# Summary

To see what variables in the dataframe mean:
https://github.com/maksimhorowitz/nflscrapR/blob/master/R/scrape_play_by_play.R

To see how to pull all the data at a player level

# Data Schema 

## Grain 

Data will be at a (game,player) grain 


# [ESPN Scoring Scheme](https://www.espn.com/fantasy/football/ffl/story?page=fflrulesstandardscoring)

## Offense:
Quarterbacks (QB), Running Backs (RB), Wide Receivers (WR), Tight Ends (TE). 
* 6 pts per rushing or receiving TD. 
* 6 pts for player returning kick/punt for TD. 
* 6 pts for player returning or recovering a fumble for TD. 
* 4 pts per passing TD. 
* 2 pts per rushing or receiving 2 pt conversion (note: teams do not receive points
for yardage gained during the conversion). 
* 2 pts per passing 2 pt conversion. 
* 1 pt per 10 yards rushing or receiving. 
* 1 pt per 25 yards passing. 

### Bonus Points
* 2 pts per rushing or receiving TD of 40 yards or more. 
* 2 pts per passing TD of 40 yards or more
(note: the player must score a touchdown to score the points)  

### Penalty Points
* -2 pts per intercepted pass. 
* -2 pts per fumble lost. 

# [Variable Dictionary (Play by Play Data)](https://github.com/maksimhorowitz/nflscrapR/blob/master/R/scrape_play_by_play.R)

## Notes 

All variables ending in `player_id` have equivalent `player_name` versions

## Game Level Information 

* game_id : ten digit id for nfl game 
* posteam : team with posession 

## Play Level Information 

* play_type : pass, run, punt, field_goal, kickoff, extra_point, qb_kneel, qb_spike, no_play (timeouts/penalties), missing 
* penalty : binary indicator if penalty on play 
* fumble : binary indicator if fumble occurred 
* fumble_lost : binary indicator if fumble lost 

These are simple variables to calculate ff points for TEs, RBs and WRs

* yards_gained : numeric yards gained (or lost) for given play 
* touchdown : binary if touchdown 

Specific variables for different types are below:

## Passing 

* receiver_player_id : unique id for player targeted on pass
* pass_touchdown : binary if passing td 
* complete_pass : binary if pass completed   
* air_yards : distance in yards perpendicular to line of scrimmaage
* yards_after_catch 
* incomplete_pass : binary if pass incomplete 
* interception : binary is pass was intercepted 
* pass_attempt : binary if pass attempt 

## Running 

* rusher_player_id 
* rush_attempt : binary if rush attempt 
* rush_touchdown : binary if rushing td 

## Laterals 

* lateral_receiver_player_id 
* lateral_rusher_player_id 
* lateral_reception : binary 
* lateral_rush : binary
* lateral_return : binary 

## Special Teams 

* punt_returner_player_id 
* kickoff_returner_player_id
* lateral_kickoff_returner_player_id
* return_touchdown : binary if return td 
* return_team : string abbrev of return team 
* return_yards : return yards gained by the return team 

## Other 

* two_point_conv_result : success, failure, safety or return 



# Analysis Tips

## Passing 

* To get length of incomplete passes -> `complete_pass == False` and sum over `air_yards` 

## Kick Returns

* Kickoff returns the return team is the team in possession `posteam`, whereas in punt returns the return team is the defense team `defteam`

# Aggregated SQL Database : TEs, RBs, WRs

## Table Name = offenseNonQB

### Grain of Data 
* game_id   
* player_id 

### Player/Team of Player Info
* full_player_name
* position
* team_name
* team_score_player
* team_score_opp
* home_game_flag

### What season/when is it in Season
* season_year
* week_of_season
* game_of_season

### Aggregated Player Stats 
* yards_gained_total
* touchdown_total
* two_point_conv_total
* touches_total
* fumble_total 
* fumble_lost_total 

### Rushing Player Stats
* yards_gained_rush 
* touchdown_rush
* fumble_rush
* fumble_lost_rush
* attempts

### Receiving Player Stats
* yards_gained_rec
* touchdown_rec
* fumble_rec
* fumble_lost_rec
* targets
* receptions
* air_yards
* yards_after_catch

### Two Point Conversion Player Stats
* two_point_conv_rush
* two_point_conv_rec

### Kickoff Return Player Stats
* yards_gained_kickoff
* touchdown_kickoff
* fumble_kickoff
* fumble_lost_kickoff
* kickoff_return_attempt

### Punt Return Player Stats
* yards_gained_punt
* touchdown_punt
* fumble_punt
* fumble_lost_punt 
* punt_return_attempt
       

In [202]:
import os
import math
import pandas as pd 
import sqlite3

# Paths
path_dict = {}
path_dict['root'] = '/Users/ryanstevens/Documents/github/nflscrapR-data/'
path_dict['games_data'] = os.path.join(path_dict['root'],'games_data')
path_dict['pbp_data'] = os.path.join(path_dict['root'],'play_by_play_data')
path_dict['roster_data'] = os.path.join(path_dict['root'],'roster_data')
path_dict['sql_data'] = os.path.join(path_dict['root'],'sql_data')



In [59]:


# Returns specific type of data (pbp, games etc) for given
#  year + season combo
def get_data(year
            ,path_dict 
            ,type='pbp'
            ,season='regular'):
    
    if season == 'regular':
        season_short = 'reg'
    else:
        season_short = season 

    # Path to data 
    path_to_data = (os.path.join(path_dict['{type}_data'.format(type=type)]
                              ,'{season}_season'
                              ,'{season_short}_{type}_{year}.csv')
                          .format(type=type
                                  ,year=year
                                  ,season=season
                                  ,season_short=season_short))
    
    df = pd.read_csv(path_to_data)

    return(df)


# Merge play by play data + roster data for given year
def pbp_with_rosters(year
                    ,path_dict 
                    ,season='regular'):
    
    # PBP data
    pdp = get_data(year,path_dict)

    # Roster data
    roster = get_data(year,path_dict,type='roster')

    # Turn roster data into dictionary
    roster_dict = dict(zip(roster['gsis_id'],roster['position']))

    # Get any columns with player_id
    player_id_cols = [col for col in pdp.columns if re.search('player_id',col) is not None]

    for player_id_col in player_id_cols:
        pdp['{0}_pos'.format(player_id_col)] = [roster_dict[elem] if (isinstance(elem,str) and elem in roster_dict.keys()) else '' for elem in pdp[player_id_col]]

    return(pdp)


In [254]:
## SQL Info
##----------

# Path to claim db
path_to_sql_db = os.path.join(path_dict['sql_data'],'analysis_data_v0.db')

# Create connection + cursor
sql_connect = sqlite3.connect(path_to_sql_db)
cursor = sql_connect.cursor()

## Create tables
sql_query = '''CREATE TABLE offenseNonQB(
game_id INTEGER, 
player_id TEXT,
full_player_name TEXT,
position TEXT,
team_name TEXT,
team_score_player TEXT,
team_score_opp INTEGER,
home_game_flag INTEGER,
season_year INTEGER,
week_of_season INTEGER,
game_of_season INTEGER,
yards_gained_total INTEGER,
touchdown_total INTEGER,
two_point_conv_total INTEGER,
touches_total INTEGER,
fumble_total INTEGER,
fumble_lost_total INTEGER,
yards_gained_rush INTEGER, 
touchdown_rush INTEGER,
fumble_rush INTEGER,
fumble_lost_rush INTEGER,
attempts INTEGER,
yards_gained_rec INTEGER,
touchdown_rec INTEGER,
fumble_rec INTEGER,
fumble_lost_rec INTEGER,
targets INTEGER,
receptions INTEGER,
air_yards INTEGER,
yards_after_catch INTEGER,
two_point_conv_rush INTEGER,
two_point_conv_rec INTEGER,
yards_gained_kickoff INTEGER,
touchdown_kickoff INTEGER,
fumble_kickoff INTEGER,
fumble_lost_kickoff INTEGER,
kickoff_return_attempt INTEGER,
yards_gained_punt INTEGER,
touchdown_punt INTEGER,
fumble_punt INTEGER,
fumble_lost_punt INTEGER, 
punt_return_attempt INTEGER,
UNIQUE(game_id, player_id)
)
'''
cursor.execute(sql_query)
sql_connect.commit()

## Create query to insert data into table
insert_data_query, sql_col_names = create_sql_insert_query(cursor,'offenseNonQB')

In [218]:
def insert_bulk_data_into_table(connect
                         ,data
                         ,sql_query):
    # Cursor
    c = connect.cursor()
    # Put data in tuple of values
    #  to be compatible with sqlite format
    tuples = [tuple(x) for x in data.to_numpy()]
    # Execute and commit
    c.executemany(sql_query,tuples)

def create_sql_insert_query(cursor,tbl_name):
    # Get columns names
    cursor.execute('PRAGMA TABLE_INFO({0})'.format(tbl_name))
    col_info = {x[0]:x[1] for x in map(lambda x: (x[1],x[2]), cursor.fetchall())}
    col_names = '({0})'.format(','.join(list(col_info.keys())))

    # Create sql query
    sql_query="INSERT INTO {tbl_name} VALUES ({num_data_cols})".format(tbl_name=tbl_name
                                                         ,num_data_cols=','.join(['?']*len(col_info)))

    return((sql_query,list(col_info.keys())))

In [255]:

for year in range(2009,2020):
    print('Processing {year}'.format(year=year))

    # Regular season play by play data
    reg_season_pdp = pbp_with_rosters(year,path_dict)

    # Drop timeouts, penalties
    reg_season_pdp = reg_season_pdp.query(" play_type not in ['no_play'] ")

    # By player type, calculate stats:
    #
    # -Yards Gained
    # -Touchdowns
    # -Fumbles 
    # -Fumbles Lost
    # 
    # Do this for only TEs, WRs and RBs 

    # Receiving stats
    receiving = (reg_season_pdp.query(" receiver_player_id_pos in ['TE','WR','RB'] and complete_pass == 1.0 ")
                            .groupby(['game_id','receiver_player_id'])[['yards_gained'
                                                                    ,'touchdown'
                                                                    ,'fumble'
                                                                    ,'fumble_lost'
                                                                    ,'air_yards'
                                                                    ,'yards_after_catch']]
                        .sum()
                        .reset_index())

    # Add targets (passes not completed + completed)
    # and receptions (passes completed) 
    targets = (reg_season_pdp.query(" receiver_player_id_pos in ['TE','WR','RB'] and pass_attempt == 1.0 ")
                            .groupby(['game_id','receiver_player_id'])
                            .size()
                            .reset_index()
                            .rename(columns={0:'targets'}))
    receptions = (reg_season_pdp.query(" receiver_player_id_pos in ['TE','WR','RB'] and complete_pass == 1.0 ")
                            .groupby(['game_id','receiver_player_id'])
                            .size()
                            .reset_index()
                            .rename(columns={0:'receptions'}))
    receiving = pd.merge(receptions
                        ,receiving
                        ,on=['game_id','receiver_player_id'])
    receiving = pd.merge(targets
                        ,receiving
                        ,on=['game_id','receiver_player_id']
                        ,how='outer')
    receiving.fillna(0,inplace=True)

    receiving = receiving.rename(columns={'receiver_player_id':'player_id'})


    # Rushing stats
    # --------------
    rushing = (reg_season_pdp.query(" rusher_player_id_pos in ['TE','WR','RB'] ")
                            .groupby(['game_id','rusher_player_id'])[['yards_gained'
                                                                        ,'touchdown'
                                                                        ,'fumble'
                                                                        ,'fumble_lost']]
                        .sum()
                        .reset_index())

    ## Add rushing attempts
    attempts = (reg_season_pdp.query(" rusher_player_id_pos in ['TE','WR','RB']  ")
                            .groupby(['game_id','rusher_player_id'])
                            .size()
                            .reset_index()
                            .rename(columns={0:'attempts'}))

    rushing = pd.merge(rushing
                    ,attempts
                    ,on=['game_id','rusher_player_id'])

    rushing = rushing.rename(columns={'rusher_player_id':'player_id'})



    # Two point conversions
    # ----------------------
    two_point_conv_rushing = (reg_season_pdp.query(" rusher_player_id_pos in ['TE','WR','RB'] and two_point_conv_result == 'success' ")
                                .groupby(['game_id','rusher_player_id'])
                                .size()
                                .reset_index()
                                .rename(columns={0:'two_point_conv'
                                                    ,'rusher_player_id':'player_id'}))

    two_point_conv_receiving = (reg_season_pdp.query(" rusher_player_id_pos in ['TE','WR','RB'] and two_point_conv_result == 'success' ")
                                .groupby(['game_id','receiver_player_id'])
                                .size()
                                .reset_index()
                                .rename(columns={0:'two_point_conv'
                                                ,'receiver_player_id':'player_id'}))
    two_point_conv = pd.merge(two_point_conv_rushing
                            ,two_point_conv_receiving
                            ,on=['player_id','game_id']
                            ,suffixes=['_rush','_rec'])

    # Kickoff + Punt Returns
    # -----------------------
    kickoff_returns = (reg_season_pdp.query(" kickoff_returner_player_id_pos in ['TE','WR','RB'] ")
                                    .groupby(['game_id','kickoff_returner_player_id'])[['yards_gained'
                                                                                            ,'touchdown'
                                                                                            ,'fumble'
                                                                                            ,'fumble_lost']]
                        .sum()
                        .reset_index()
                        .rename(columns={'kickoff_returner_player_id':'player_id'}))

    kickoff_attempts = (reg_season_pdp.query(" kickoff_returner_player_id_pos in ['TE','WR','RB'] ")
                                .groupby(['game_id','kickoff_returner_player_id'])
                                .size()
                                .reset_index()
                                .rename(columns={0:'kickoff_return_attempt'
                                                ,'kickoff_returner_player_id':'player_id'}))

    kickoff_returns = pd.merge(kickoff_returns
                            ,kickoff_attempts
                            ,on=['player_id','game_id'])
                    

    punt_returns = (reg_season_pdp.query(" punt_returner_player_id_pos in ['TE','WR','RB'] ")
                                    .groupby(['game_id','punt_returner_player_id'])[['yards_gained'
                                                                                            ,'touchdown'
                                                                                            ,'fumble'
                                                                                            ,'fumble_lost']]
                        .sum()
                        .reset_index()
                        .rename(columns={'punt_returner_player_id':'player_id'}))

    punt_attempts = (reg_season_pdp.query(" punt_returner_player_id_pos in ['TE','WR','RB'] ")
                                .groupby(['game_id','punt_returner_player_id'])
                                .size()
                                .reset_index()
                                .rename(columns={0:'punt_return_attempt'
                                                ,'punt_returner_player_id':'player_id'}))

    punt_returns = pd.merge(punt_returns
                            ,punt_attempts
                            ,on=['player_id','game_id'])  

    ## Combine kickoff + punt returns data
    returns = pd.merge(kickoff_returns
                    ,punt_returns
                    ,on=['player_id','game_id']
                    ,how='outer'
                    ,suffixes=['_kickoff','_punt'])     
    returns.fillna(0,inplace=True)   


    ##------------------
    ## TOTAL STATISTICS
    ##------------------


    # Combining rushing, receiving, two point conversions + special teams returns 
    offense_non_qb = pd.merge(rushing 
                            ,receiving
                            ,on=['game_id','player_id']
                            ,how='outer'
                            ,suffixes=['_rush','_rec'])

    offense_non_qb = pd.merge(offense_non_qb 
                            ,two_point_conv
                            ,on=['game_id','player_id']
                            ,how='outer')

    offense_non_qb = pd.merge(offense_non_qb 
                            ,returns
                            ,on=['game_id','player_id']
                            ,how='outer'
                            ,suffixes=['','_returns'])
    offense_non_qb.fillna(0,inplace=True)


    # Calculate total game level statistics 
    #   
    #  Yards Gained 
    #  Touchdowns
    #  Two Point Conversions
    #  Touches (pass attempts + rushing attempts)  
    offense_non_qb['yards_gained_total'] = offense_non_qb[['yards_gained_rush','yards_gained_rec','yards_gained_kickoff','yards_gained_punt']].sum(axis=1)
    offense_non_qb['touchdown_total'] = offense_non_qb[['touchdown_rush','touchdown_rec','touchdown_kickoff','touchdown_punt']].sum(axis=1)
    offense_non_qb['fumble_total'] = offense_non_qb[['fumble_rush','fumble_rec','fumble_kickoff','fumble_punt']].sum(axis=1)
    offense_non_qb['fumble_lost_total'] = offense_non_qb[['fumble_lost_rush','fumble_lost_rec','fumble_lost_kickoff','fumble_lost_punt']].sum(axis=1)
    offense_non_qb['two_point_conv_total'] = offense_non_qb[['two_point_conv_rush','two_point_conv_rec']].sum(axis=1)
    offense_non_qb['touches_total'] = offense_non_qb[['attempts','targets']].sum(axis=1)


    # Add players full name name 
    rosters = get_data(year,path_dict,type='roster')
    offense_non_qb = pd.merge(offense_non_qb
                            ,rosters[['full_player_name','position','gsis_id']]
                            ,left_on=['player_id']
                            ,right_on=['gsis_id'])

    # Add team player is playing for in specific game 
    receiver_teams=(reg_season_pdp.query(" receiver_player_id_pos in ['TE','WR','RB'] ")[['receiver_player_id','posteam','game_id']]
                .drop_duplicates()
                .rename(columns={'receiver_player_id' : 'player_id'
                                ,'posteam':'team_name'}))
    rusher_teams=(reg_season_pdp.query(" rusher_player_id_pos in ['TE','WR','RB'] ")[['rusher_player_id','posteam','game_id']]
                .drop_duplicates()
                .rename(columns={'rusher_player_id' : 'player_id'
                                ,'posteam':'team_name'}))

    kickoff_returner_teams=(reg_season_pdp.query(" kickoff_returner_player_id_pos in ['TE','WR','RB'] ")[['kickoff_returner_player_id','posteam','game_id']]
                .drop_duplicates()
                .rename(columns={'kickoff_returner_player_id' : 'player_id'
                                ,'posteam':'team_name'}))

    punt_returner_teams=(reg_season_pdp.query(" punt_returner_player_id_pos in ['TE','WR','RB'] ")[['punt_returner_player_id','defteam','game_id']]
                .drop_duplicates()
                .rename(columns={'punt_returner_player_id' : 'player_id'
                                ,'defteam':'team_name'}))

    player_to_team = (receiver_teams.append([rusher_teams,kickoff_returner_teams,punt_returner_teams])
                                    .drop_duplicates())
    offense_non_qb = pd.merge(offense_non_qb
                            ,player_to_team
                            ,on=['player_id','game_id']
                            ,how='left')
    offense_non_qb['team_name'].fillna('',inplace=True)

    ## Which game of the season is it
    games = get_data(year,path_dict,type='games')

    # Seperate home + away scores, creating
    #  flag for if team is home team
    home_games = games[['home_team','week','home_score','away_score','game_id']].rename(columns={'home_team':'team_name'
                                                                                    ,'home_score':'team_score_player'
                                                                                    ,'away_score':'team_score_opp'})
    home_games['home_game_flag'] = True
    away_games = games[['away_team','week','home_score','away_score','game_id']].rename(columns={'away_team':'team_name'
                                                                                    ,'home_score':'team_score_opp'
                                                                                    ,'away_score':'team_score_player'})
    away_games['home_game_flag'] = False
    games_scores = home_games.append(away_games)

    # Create game of season variable, 
    #  which is the actual number of games 
    #  a team has played up until that week
    games_scores.sort_values(['team_name','week'],inplace=True)
    games_scores['counter']=1
    games_scores['game_of_season'] = (games_scores.groupby(['team_name'])['counter']
                                                .cumsum())
    games_scores.rename(columns={'week':'week_of_season'},inplace=True)
    games_scores.drop('counter',axis=1,inplace=True)

    # Add to main data
    offense_non_qb = pd.merge(offense_non_qb
                            ,games_scores 
                            ,on=['team_name','game_id'])

    # Delete unncessary columns
    offense_non_qb.drop(['gsis_id'],axis=1,inplace=True)

    # Create season_year variable to keep track of season
    offense_non_qb['season_year'] = year



    ## --------------------------
    ## ADD DATA TO SQL DATABASE
    ## --------------------------

    # Order data 
    cols_in_data = offense_non_qb.columns
    offense_non_qb = offense_non_qb[sql_col_names]

    # Put data in tuple of values
    #  to be compatible with sqlite format
    tuples = [tuple(x) for x in offense_non_qb.to_numpy()]
    # Execute and commit
    cursor.executemany(insert_data_query,tuples)
    sql_connect.commit()
                        

Processing 2009
Processing 2010
Processing 2011
Processing 2012
Processing 2013
Processing 2014
Processing 2015
Processing 2016
Processing 2017
Processing 2018
Processing 2019


For TEs, RBs and WRs, we split up rushing, receiving, and special teams info:

* Yards Gained 
* Air Yards Gained (if reception)
* Touchdown 
* 2 point conversion 
* Fumbles 
* Fumbles Lost 

6 pts per rushing or receiving TD.
6 pts for player returning kick/punt for TD.
6 pts for player returning or recovering a fumble for TD.
4 pts per passing TD.
2 pts per rushing or receiving 2 pt conversion (note: teams do not receive points for yardage gained during the conversion).
2 pts per passing 2 pt conversion.
1 pt per 10 yards rushing or receiving.
1 pt per 25 yards passing.